In [ ]:
# Week 6 Homework: Data Retrieval and Processing

# This script is intended to be run in a Jupyter Notebook environment.
# To run this script, ensure you have Jupyter Notebook installed and running.

# You can install Jupyter Notebook using pip:
# %pip install notebook

# You can start Jupyter Notebook by running the following command in your terminal:
# $ jupyter notebook 

# This script connects to a local Ganache blockchain using Web3.py.
# Make sure you have Ganache running and Web3.py installed.
# %pip install web3

from web3 import Web3
import json, time


# Load CSV file from your Homework 1
import pandas as pd


# Load IoT sensor data from CSV file (Generated in Homework 1)
iot_df = pd.read_csv('Week 6 Homework/iot_data.csv')

# Display the first few rows of the DataFrame
print(iot_df.head())

# Connect to local Ganache blockchain
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# Check if connected to Ganache
if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Ensure Ganache is running.")
    exit()

# Replace with actual contract address form Remix 
contract_address = "0xd031B8de5e372d5c40040c93e387c4240350B7bF"

# Paste the ABI from Remix 
# abi = [...] # Replace with the actual ABI of your contract

# If you have the ABI in a file, you can load it like this:
with open('Week 6 Homework/copy-of-abi.json', 'r') as abi_file:
    abi = json.load(abi_file)

# Load the smart contract
contract = web3.eth.contract(address=contract_address, abi=abi)

# Set the default sender address (the first account from Ganache)
web3.eth.default_account = web3.eth.accounts[0]

print(f"✅ Connected to Smart Contract at {contract_address}")


# Send the IoT data to the blockchain. Each row from the CSV file will be stored as a transaction on the blockchain.
def send_iot_data(package_id, gps_coords, temperature, status):
    
    txn = contract.functions.storeData(
        package_id,
        gps_coords,
        temperature,
        status
    ).transact({
        'from': web3.eth.default_account,
        'gas': 3000000
    })
    
    receipt = web3.eth.wait_for_transaction_receipt(txn)
    print(f"✅ Data Stored {package_id} GPS: {gps_coords:>23} Temp: {temperature}  Status: {status:<10} at block {
        receipt.blockNumber:>3} Txn hash {receipt.transactionHash.hex()}")


# Iterate through the DataFrame and send each row to the blockchain
for index, row in iot_df.iterrows():

    package_id  = str(row['package_id'  ])
    gps_coords  = str(row['gps_coords'  ])
    temperature = str(row['temperature' ])
    status      = str(row['status'      ])    

    send_iot_data(package_id, gps_coords, temperature, status)    
    time.sleep(1)  # sleep to avoid hitting the gas limit too quickly

# now that the data is on the blockchain, we can retrieve it to verify storage
total_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_records}")

# retrieve and print a specific record (e.g., the first one)
record = contract.functions.getRecord(0).call()
print(f"First Stored Record : {record}")

               timestamp package_id               gps_coords temperature  \
0  2025-06-02T21:36:32I0     PKG001   (7.839114, 125.820784)        29 C   
1  2025-06-02T21:36:32I1     PKG002  (16.754497, 116.599905)        24 C   
2  2025-06-02T21:36:32I2     PKG003  (16.521005, 119.641554)        29 C   
3  2025-06-02T21:36:32I3     PKG004  (10.941567, 118.303819)        29 C   
4  2025-06-02T21:36:32I4     PKG005   (9.844815, 118.509812)        29 C   

       status  
0   Delivered  
1   Delivered  
2     Delayed  
3   Delivered  
4  In Transit  
✅ Connected to Ganache successfully!
✅ Connected to Smart Contract at 0xd031B8de5e372d5c40040c93e387c4240350B7bF
✅ Data Stored PKG001 GPS:  (7.839114, 125.820784) Temp: 29 C  Status: Delivered  at block   2 Txn hash b6ca0f7550a36ec2952550945f9317c1e6f1df88b9a456de26413f565178f86d
✅ Data Stored PKG002 GPS: (16.754497, 116.599905) Temp: 24 C  Status: Delivered  at block   3 Txn hash df5fd87cb1bce331f33984546db4cf3ffd6a4a72eb02e4908d9e3288dea0a7c

In [20]:
total_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_records}")

Total IoT records stored: 100


In [21]:
import pandas as pd
import numpy as np

# Retrieve all IoT records from the blockchain
iot_records = [] 

# Convert the stored data back to a DataFrame for easy viewing
total_records = contract.functions.getTotalRecords().call()    
for i in range(total_records):
    record = contract.functions.getRecord(i).call()
    iot_records.append({
        "timestamp"  : record[0],
        "package_id" : record[1],
        "gps_coords" : record[2],
        "temperature": record[3],
        "status"     : record[4]
    })
        
# Covert to a DataFrame
iot_df = pd.DataFrame(iot_records)

# Convert timestamp to a readable format
iot_df['timestamp'] = pd.to_datetime(iot_df['timestamp'], unit='s')

# Display the DataFrame
print(iot_df.head())

            timestamp package_id               gps_coords temperature  \
0 2025-06-09 07:41:56     PKG001   (7.839114, 125.820784)        29 C   
1 2025-06-09 07:41:57     PKG002  (16.754497, 116.599905)        24 C   
2 2025-06-09 07:41:58     PKG003  (16.521005, 119.641554)        29 C   
3 2025-06-09 07:41:59     PKG004  (10.941567, 118.303819)        29 C   
4 2025-06-09 07:42:00     PKG005   (9.844815, 118.509812)        29 C   

       status  
0   Delivered  
1   Delivered  
2     Delayed  
3   Delivered  
4  In Transit  


In [22]:
import numpy as np


'''
Longitude and latitude are the coordinates used to pinpoint locations on Earth's surface. 
They create a grid system that allows precise positioning.

Latitude - measures how far north or south a place is from the Equator. 
    It’s expressed in degrees (°) from 0° at the Equator up to 90° at the poles.

Longitude - measures how far east or west a place is from the Prime Meridian, which is 0° longitude. 
    It extends up to 180° east or west.

Example for the Philippines:
Metro Manila, the capital region, has coordinates approximately:
Latitude: 14.5995° N (north of the Equator)
Longitude: 120.9842° E (east of the Prime Meridian)

Every place in the world has a unique set of longitude and latitude coordinates—kind of like a GPS address for locations! 

The coordinates 7.839114, 125.820784 are in "decimal degrees (DD)" format, and 
they have been converted into "degrees, minutes, and seconds (DMS)" format. 

Here’s how the conversion works:

1. Latitude: 7.839114° N
   - The whole number 7 is the degrees (°).
   - Multiply the decimal part (0.839114) by 60 to get minutes:
     - 0.839114 × 60 = 50.347 → 50 minutes (keep the decimal part for seconds).
   - Multiply the remaining decimal (0.347) by 60 to get seconds:
     - 0.347 × 60 = 20.8 → 20.8 seconds.
   - Final latitude in DMS: 7°50'20.8" N.

2. Longitude: 125.820784° E
   - The whole number 125 is the degrees (°).
   - Multiply the decimal part (0.820784) by 60 to get minutes:
     - 0.820784 × 60 = 49.247 → 49 minutes.
   - Multiply the remaining decimal (0.247) by 60 to get seconds:
     - 0.247 × 60 = 14.8 → 14.8 seconds.
   - Final longitude in DMS: 125°49'14.8" E.

So, 7.839114, 125.820784 (DD) → 7°50'20.8" N, 125°49'14.8" E (DMS). 

This conversion is useful when working with traditional navigation systems or maps. 
'''

# Extract numeric values from 'data_value' where applicable
iot_df["temp_in_celsius"] = iot_df["temperature"].str.extract(r'(\d+\.?\d*)').astype(float)
yx_coords = iot_df["gps_coords"].str.split(', ', expand=True)
print(yx_coords)

# Extract latitude and longitude from 'gps_coords'
iot_df["latitude"] = iot_df["gps_coords"].str.extract(r'(\d+\.?\d*)').astype(float)
iot_df["longitude"] = iot_df["gps_coords"].str.extract(r'(\d+\.?\d*)').astype(float)


# Handle missing values (if any)
iot_df.fillna(0, inplace=True)


# Display cleaned data
print(iot_df.head())

             0            1
0    (7.839114  125.820784)
1   (16.754497  116.599905)
2   (16.521005  119.641554)
3   (10.941567  118.303819)
4    (9.844815  118.509812)
..         ...          ...
95  (12.998725  125.744701)
96  (20.336235   126.65279)
97   (8.659069  126.519407)
98   (9.128521  123.681135)
99   (8.911339  126.627085)

[100 rows x 2 columns]
            timestamp package_id               gps_coords temperature  \
0 2025-06-09 07:41:56     PKG001   (7.839114, 125.820784)        29 C   
1 2025-06-09 07:41:57     PKG002  (16.754497, 116.599905)        24 C   
2 2025-06-09 07:41:58     PKG003  (16.521005, 119.641554)        29 C   
3 2025-06-09 07:41:59     PKG004  (10.941567, 118.303819)        29 C   
4 2025-06-09 07:42:00     PKG005   (9.844815, 118.509812)        29 C   

       status  temp_in_celsius   latitude  longitude  
0   Delivered             29.0   7.839114   7.839114  
1   Delivered             24.0  16.754497  16.754497  
2     Delayed             29.0  16.5

In [23]:
# Save cleaned IoT data to a CSV file
iot_df.to_csv("cleaned_iot_data.csv", index=False)


print("✅ Cleaned IoT data saved successfully as cleaned_iot_data.csv")

✅ Cleaned IoT data saved successfully as cleaned_iot_data.csv
